In [35]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
from scipy.stats import zscore

import pymanopt
from pymanopt.manifolds import Stiefel
from pymanopt.optimizers import TrustRegions

#### LOAD DATA

In [26]:
# % this data is from an example session:
# %   brain region - Left ALM
# %   task - delayed response (DR) + water-cued (WC)
# %   

# % exampleData is a dict containing key/vals:
# %   time - time bins corresponding to first dimension of fields 'seq' and
# %          'motionEnergy' (aligned to go cue/water drop)
# %   seq  - single trial neural firing rates in 5 ms bins and smoothed with
# %          causal gaussian kernel (35 ms s.d.). Shape = (time bins, trials,
# %          neurons)
# %   motionEnergy - motion energy is used to measure amount of movement
# %                  at a given frame. The time series has been resampled to 
# %                  match neural data. Shape = (time bins, trials)
# %   moveMask - logical array of shape (time bins, trials). 0 indicates
# %              stationarity, 1 indicates moving. This mask was produced from
# %              exampleData.motionEnergy using a manually set threshold
# %   anm/date - session meta data

data = loadmat('exampleData.mat')

exampleData = dict()
exampleData['anm'] = data['exampleData'][0][0][0] 
exampleData['date'] = data['exampleData'][0][0][1] 
exampleData['time'] = data['exampleData'][0][0][2] 
exampleData['seq'] = data['exampleData'][0][0][3]
exampleData['motionEnergy'] = data['exampleData'][0][0][4]
exampleData['moveMask'] = data['exampleData'][0][0][5]


nBins = exampleData['seq'].shape[0]
nTrials = exampleData['seq'].shape[1]
nNeurons = exampleData['seq'].shape[2]


In [27]:
def create_cost_and_egrad(manifold, eigvalsNull, eigvalsPotent, dNull, dPotent, P1, P2, covNull, covPotent):
    egrad = None # euclidean gradient

    @pymanopt.function.numpy(manifold)
    def cost(Q):
        Qpotent = Q @ P1
        Qnull = Q @ P2
        normPotent = np.sum(eigvalsPotent[1:dPotent])
        normNull = np.sum(eigvalsNull[1:dNull])
        return -0.5 * np.trace(Qpotent.T @ covPotent @ Qpotent) / normPotent - 0.5 * np.trace(Qnull.T @ covNull @ Qnull) / normNull


    @pymanopt.function.numpy(manifold)
    def euclidean_gradient(Q):
        normPotent = np.sum(eigvalsPotent[1:dPotent])
        normNull = np.sum(eigvalsNull[1:dNull])
        return -covPotent @ Q @ (P1*P1.T) / normPotent - covNull @ Q @ (P2*P2.T) / normNull

    return cost, egrad

In [46]:
## PREPROCESS DATA
# choice of normalization/standardization is up to you, here just zscoring

temp = exampleData['seq'].reshape((nBins*nTrials,nNeurons))

rez = dict() # input data, parameters, and results dictionary
rez['N'] = dict() # input data

rez['N']['full_cat'] = zscore(temp, axis=0) # (time bins * nTrials, nNeurons)
rez['N']['full'] = rez['N']['full_cat'].reshape((nBins,nTrials,nNeurons)); # (time bins, nTrials, nNeurons)



In [73]:
## DEFINE DATA TO USE FOR NULL AND POTENT SUBSPACES

# for the null subspace, we will use all time points, from all trials, in
# which the animal was stationary.
moveMask = exampleData['moveMask'][:].reshape(-1) # (time bins * nTrials)
rez['N']['null'] = rez['N']['full_cat'][np.logical_not(moveMask),:]

# for the potent subspace, we will use all time points, from all trials, in
# which the animal was moving.
rez['N']['potent'] = rez['N']['full_cat'][np.logical_not(np.logical_not(moveMask)),:]


In [79]:
## COVARIANCE AND DIMENSIONALITY

# covariances
rez['covNull'] = np.cov(rez['N']['null'].T)
rez['covPotent'] = np.cov(rez['N']['potent'].T)

# dimensionality of subspaces
# here I am hard-coding the number of dimensions to be 7 for both subspaces. However, one
# could perform further dimensionality reduction or keep more dimensions or have a different number of dimensions for each subspace.
# In our experience, dimensionality >  20 takes an incredibly long time to 
# optimize over.

rez['dNull'] = 7
rez['dPotent'] = 7

rez['dMax'] = np.max([rez['dNull'], rez['dPotent']])


In [ ]:
optimizer = TrustRegions(verbosity=0)
manifold = Stiefel(nNeurons,rez['dNull'] + rez['dPotent'])

## TODO
# assert(isequal(C1, C1'));
# assert(isequal(C2, C2'));

# assert(size(C1,1) == size(C2,1));
# n = size(C1,1);
# dmax = max(d1,d2);
# %largest magnitude eigenvalues
# eigvals1 = eigs(C1, dmax, 'la'); %hoping that these all eigs are positive, still only divinding by largest algebraic +ve values
# eigvals2 = eigs(C2, dmax, 'la');
# assert(~any(eigvals1<0), 'eigvals1 <0');
# assert(~any(eigvals2<0), 'eigvals2 <0');
# P1 = [eye(d1); zeros(d2,d1)];
# P2 = [zeros(d1, d2); eye(d2)];
# P = {P1,P2};


cost, egrad = create_cost_and_egrad(manifold, eigvalsNull, eigvalsPotent, dNull, dPotent, P1, P2, covNull, covPotent)

problem = pymanopt.Problem(
    manifold,
    cost,
    euclidean_gradient = egrad,
    euclidean_hessian = None,
)

Q = optimizer.run(problem).point